In [1]:
import pylhe
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import pandas as pd

In [2]:
lhe_file = "/users/eleves-b/2022/baptiste.barthe-gold/Documents/Comput_HEP/MG5_aMC_v2_9_22/VBF-SM/Events/run_02/unweighted_events.lhe.gz"
events = pylhe.read_lhe_with_attributes(lhe_file)
nevents = pylhe.read_num_events(lhe_file)
print(f"Number of events: {nevents}")

Number of events: 100000


In [3]:
def extract_4m(events):
    cross_section_weight, quarks_4m, leptons_4m = [], [], []

    for event in events:
        # select only final state particles
        part = [i for i in event.particles if i.status == 1.0]

        # select only quarks up to charm
        quarks = [i for i in part if abs(i.id) <= 4]

        # select only leptons
        leptons = [i for i in part if abs(i.id) in [11, 13, 15]]

        # sanity check, we expect quarks and leptons to be a list of two elements
        assert len(quarks) == 2, f"Length of selected quarks is not 2: {len(quarks)}"
        assert len(leptons) == 2, f"Length of selected leptons is not 2: {len(leptons)}"

        quarks_4m.append(
            [
                {
                    "px": quarks[0].px,
                    "py": quarks[0].py,
                    "pz": quarks[0].pz,
                    "e": quarks[0].e,
                },
                {
                    "px": quarks[1].px,
                    "py": quarks[1].py,
                    "pz": quarks[1].pz,
                    "e": quarks[1].e,
                },
            ]
        )

        leptons_4m.append(
            [
                {
                    "px": leptons[0].px,
                    "py": leptons[0].py,
                    "pz": leptons[0].pz,
                    "e": leptons[0].e,
                },
                {
                    "px": leptons[1].px,
                    "py": leptons[1].py,
                    "pz": leptons[1].pz,
                    "e": leptons[1].e,
                },
            ]
        )

        cross_section_weight.append(event.eventinfo.weight)

    quarks_4m = ak.Array(quarks_4m, with_name="Momentum4D")
    leptons_4m = ak.Array(leptons_4m, with_name="Momentum4D")

    return quarks_4m, leptons_4m, cross_section_weight

In [4]:
quarks_4m, leptons_4m, cross_section_weight = extract_4m(events)

In [5]:
dataset = pd.DataFrame(
    {
        "m_ll": (leptons_4m[:, 0] + leptons_4m[:, 1]).mass,
        "m_jj": (quarks_4m[:, 0] + quarks_4m[:, 1]).mass,
        "pt_l1": leptons_4m[:, 0].pt,
        "pt_l2": leptons_4m[:, 1].pt,
        "pt_j1": quarks_4m[:, 0].pt,
        "pt_j2": quarks_4m[:, 1].pt,
        "pt_ll": (leptons_4m[:, 0] + leptons_4m[:, 1]).pt,
        "eta_l1": leptons_4m[:, 0].eta,
        "eta_l2": leptons_4m[:, 1].eta,
        "eta_j1": quarks_4m[:, 0].eta,
        "eta_j2": quarks_4m[:, 1].eta,
        "delta_eta_jj": quarks_4m[:, 0].eta - quarks_4m[:, 1].eta,
        "delta_phi_jj": quarks_4m[:, 0].phi - quarks_4m[:, 1].phi,
    }
)

In [6]:
dataset.head()

,m_ll,m_jj,pt_l1,pt_l2,pt_j1,pt_j2,pt_ll,eta_l1,eta_l2,eta_j1,eta_j2,delta_eta_jj,delta_phi_jj
0,91.915108,817.571428,19.068266,119.288002,47.787994,174.241717,127.214313,0.601289,-0.827524,-3.112653,1.247709,-4.360361,-3.293960
1,91.554900,2404.014354,48.863130,87.367985,107.964679,177.910879,106.054348,-0.455842,0.039917,3.143773,-2.557357,5.701129,-3.725031
2,91.987715,662.842055,26.142133,128.186262,99.888379,107.257538,127.337882,2.250787,1.749627,1.845531,-1.855586,3.701117,-1.819143
3,91.241031,1242.649786,88.205101,27.274699,213.088461,244.515221,72.516153,0.011753,0.331417,1.069279,-2.251527,3.320807,2.854300
4,92.039306,487.140632,43.624652,59.307830,116.027217,164.351151,48.557695,-0.791976,-1.091582,3.552877,1.215985,2.336892,-3.176056


In [ ]:
dataset.to_csv("./data/SM_100k.csv", index=False)